In [3]:
# Final Model Prediction
import sys
sys.path.append('/UltimeTradingBot/Crypto_backtest_tools')
%cd /UltimeTradingBot/Crypto_backtest_tools

import pandas as pd
import json
import time
import numpy as np
import urllib
#import ccxt
import  ccxt  
import random
from keras.models import load_model
import os
import asyncio
import tensorflow as tf
## Garbage collection
import gc
gc.collect()    

import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))
## Gloabale variables
Normalization=None
Model_FileName='../Data/X004_15min_model87.hdf5'
Normalization_File='../Data/Moramalization_val_004w15.json'
Binance_USDT_HALAL = [
    "BTC/USDT",
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "UST/USDT",
    "SOL/USDT",
    "APE/USDT",
    "XRP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "ZIL/USDT",
    "ENS/USDT",
    "DOGE/USDT",
    "LTC/USDT",
    "EUR/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "WAVES/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ROSE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "ICP/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL

## Local Functions first
# Get list of all IDs on binance
def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]

def get_crypto_metadata(pair_list):
    Binance_USDT_HALAL=pair_list
    ids = []
    #ids = all_ids()
    for halalpair in Binance_USDT_HALAL:
    #    print( halalpair.replace('/',''))
        ids.append(halalpair.replace('/',''))
    #print(ids)
    MetaData=pd.DataFrame(ids)
    MetaData["Pair"]=Binance_USDT_HALAL
    counters=0
    for this_id in ids:
        '''
        Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
        Find launch Day of symbol in week
        Find launch minute of symbol in day
        '''

        symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
        symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
        symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
        MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

        counters += 1

        #print("Week stamp", symbol_launch_week_stamp)
        #print("Day  stamp", symbol_launch_day_stamp)
        #print("Min  stamp", symbol_launch_minute_stamp)
        print(counters,end=" ")
        #print(this_id, "launched", symbol_launch_minute_stamp )
    return MetaData
    #print("")
    
def instant_pair_data(pair="GMT/BUSD",exchange=ccxt.binance(),window=15):
    ex=exchange
    ticker = ex.fetch_ticker(pair)
    pair_current_price=ticker['info']['askPrice']
    #print(pair_current_price)

    ohlcv1m = ex.fetch_ohlcv(pair, '1m', limit=15)
    ohlcv5m = ex.fetch_ohlcv(pair, '5m', limit=15)
    ohlcv15m = ex.fetch_ohlcv(pair, '15m', limit=15)
    ohlcv1h = ex.fetch_ohlcv(pair, '1h', limit=15)
    ohlcv1d = ex.fetch_ohlcv(pair, '1d', limit=15)

    pair_data=pd.DataFrame()
    pair_data.loc[0,"price"]=float(pair_current_price)
    #minute
    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)]=ohlcv1m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)]=ohlcv1m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)]=ohlcv1m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)]=ohlcv1m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)]=ohlcv1m[-window_i][4]

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_day"]=ohlcv1d[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_day"]=ohlcv1d[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_day"]=ohlcv1d[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_day"]=ohlcv1d[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_day"]=ohlcv1d[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_hour"]=ohlcv1h[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_15min"]=ohlcv15m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_15min"]=ohlcv5m[-window_i][4]  

    for window_i in range(1,window+1):
        pair_data.loc[0,"high-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][1]
        pair_data.loc[0,"low-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][2]
        pair_data.loc[0,"open-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][0]
        pair_data.loc[0,"close-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][3]
        if(window_i!=1):pair_data.loc[0,"volume-"+str(window_i)+"_5min"]=ohlcv5m[-window_i][4]   
    return  pair_data

def normalize(dataset,file='w15_NoVol_Normalization.json'):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization=None
    if(Normalization==None):
        #print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        #print('normalization is loaded')
        pass

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)

def Buy_Dessision(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Normalized(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Multi_In_Out(input):
    A=np.array(input)
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

def Buy_Dessision_Multi_In_Out_Normalized(input):
    A=np.array(input)
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded)


/UltimeTradingBot/Crypto_backtest_tools


In [2]:

# Getting Vars

try:
    MetaData=pd.read_csv("../Data/MetaData.csv")
except:
    MetaData=get_crypto_metadata(Binance_USDT_HALAL)

## loding the model
model = load_model(Model_FileName)
Exchange=ccxt.binance()
ex=Exchange
pair='GMT/BUSD'


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 

2022-09-25 13:07:33.734200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-25 13:07:33.734237: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-25 13:07:33.734264: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ibm56go): /proc/driver/nvidia/version does not exist
2022-09-25 13:07:33.734524: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:

df=pd.read_csv("../Data/004Precent-w15_crypto_shufled_NoVol.csv")

In [ ]:
df[df["buy"]==1]